In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
def data():
    # Split train & validation set
    BBB_Train_Decile2 = pd.read_pickle("BBB_Train_Decile2.pkl")
    BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("BBB_Test_Decile2.pkl")
    y_test = pd.read_pickle("BBB_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(BBB_Train_Decile2, BBB_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150, 200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM BBB Decile 2')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))
    

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3087/3087 [==============================] - 5s 2ms/step - loss: 0.5329 - acc: 0.7438 - val_loss: 0.4982 - val_acc: 0.7668
Epoch 39/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5269 - acc: 0.7512 - val_loss: 0.4988 - val_acc: 0.7630
Epoch 40/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5321 - acc: 0.7557 - val_loss: 0.4969 - val_acc: 0.7681
Epoch 41/100
3087/3087 [==============================] - 5s 1ms/step - loss: 0.5312 - acc: 0.7567 - val_loss: 0.4962 - val_acc: 0.7565
Epoch 42/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5296 - acc: 0.7512 - val_loss: 0.4959 - val_acc: 0.7694
Epoch 43/100
3087/3087 [==============================] - 5s 1ms/step - loss: 0.5274 - acc: 0.7580 - val_loss: 0.4954 - val_acc: 0.7668
Epoch 44/100
3087/3087 [==============================] - 5s 1ms/step - loss: 0.5309 - acc: 0.7557 - val_loss: 0.4952 - val_acc: 0.7681
Epoch 45/100
3087/3087 [==============================] - 5s 

Epoch 98/100
3087/3087 [==============================] - 5s 1ms/step - loss: 0.5162 - acc: 0.7606 - val_loss: 0.4895 - val_acc: 0.7694
Epoch 99/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5182 - acc: 0.7606 - val_loss: 0.4893 - val_acc: 0.7630
Epoch 100/100
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5172 - acc: 0.7603 - val_loss: 0.4895 - val_acc: 0.7694
Test accuracy: 0.7694300518134715
Train on 3087 samples, validate on 772 samples
Epoch 1/150
3087/3087 [==============================] - 7s 2ms/step - loss: 0.6327 - acc: 0.6352 - val_loss: 0.5825 - val_acc: 0.6878
Epoch 2/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5710 - acc: 0.7117 - val_loss: 0.5157 - val_acc: 0.7500
Epoch 3/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5469 - acc: 0.7392 - val_loss: 0.5080 - val_acc: 0.7668
Epoch 4/150
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5411 - acc: 0.7493 - val_l

Epoch 56/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4884 - acc: 0.7804 - val_loss: 0.4746 - val_acc: 0.7850
Epoch 57/150
3087/3087 [==============================] - 5s 1ms/step - loss: 0.4906 - acc: 0.7820 - val_loss: 0.4746 - val_acc: 0.7850
Epoch 58/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4961 - acc: 0.7810 - val_loss: 0.4748 - val_acc: 0.7863
Epoch 59/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4910 - acc: 0.7826 - val_loss: 0.4747 - val_acc: 0.7863
Epoch 60/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4874 - acc: 0.7807 - val_loss: 0.4746 - val_acc: 0.7863
Epoch 61/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4867 - acc: 0.7817 - val_loss: 0.4745 - val_acc: 0.7850
Epoch 62/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4892 - acc: 0.7804 - val_loss: 0.4745 - val_acc: 0.7850
Epoch 63/150
3087/3087 [========================

3087/3087 [==============================] - 4s 1ms/step - loss: 0.4873 - acc: 0.7852 - val_loss: 0.4744 - val_acc: 0.7863
Epoch 117/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4847 - acc: 0.7839 - val_loss: 0.4744 - val_acc: 0.7889
Epoch 118/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4895 - acc: 0.7800 - val_loss: 0.4744 - val_acc: 0.7876
Epoch 119/150
3087/3087 [==============================] - 5s 1ms/step - loss: 0.4889 - acc: 0.7784 - val_loss: 0.4745 - val_acc: 0.7876
Epoch 120/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4875 - acc: 0.7804 - val_loss: 0.4745 - val_acc: 0.7876
Epoch 121/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4896 - acc: 0.7768 - val_loss: 0.4746 - val_acc: 0.7876
Epoch 122/150
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4864 - acc: 0.7787 - val_loss: 0.4746 - val_acc: 0.7876
Epoch 123/150
3087/3087 [==============================

...

Epoch 78/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5243 - acc: 0.7651 - val_loss: 0.4919 - val_acc: 0.7720
Epoch 79/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5204 - acc: 0.7651 - val_loss: 0.4917 - val_acc: 0.7707
Epoch 80/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5196 - acc: 0.7648 - val_loss: 0.4916 - val_acc: 0.7681
Epoch 81/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5212 - acc: 0.7681 - val_loss: 0.4922 - val_acc: 0.7694
Epoch 82/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5180 - acc: 0.7674 - val_loss: 0.4920 - val_acc: 0.7694
Epoch 83/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5245 - acc: 0.7651 - val_loss: 0.4918 - val_acc: 0.7707
Epoch 84/100
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5192 - acc: 0.7674 - val_loss: 0.4914 - val_acc: 0.7707
Epoch 85/100
3087/3087 [========================

Epoch 36/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4969 - acc: 0.7755 - val_loss: 0.4806 - val_acc: 0.7772
Epoch 37/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.5003 - acc: 0.7713 - val_loss: 0.4806 - val_acc: 0.7759
Epoch 38/200
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4947 - acc: 0.7726 - val_loss: 0.4807 - val_acc: 0.7772
Epoch 39/200
3087/3087 [==============================] - 5s 1ms/step - loss: 0.5036 - acc: 0.7755 - val_loss: 0.4807 - val_acc: 0.7772
Epoch 40/200
3087/3087 [==============================] - 5s 1ms/step - loss: 0.5005 - acc: 0.7765 - val_loss: 0.4808 - val_acc: 0.7772
Epoch 41/200
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5014 - acc: 0.7723 - val_loss: 0.4807 - val_acc: 0.7759
Epoch 42/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4954 - acc: 0.7703 - val_loss: 0.4808 - val_acc: 0.7759
Epoch 43/200
3087/3087 [========================

3087/3087 [==============================] - 5s 2ms/step - loss: 0.4994 - acc: 0.7723 - val_loss: 0.4806 - val_acc: 0.7746
Epoch 97/200
3087/3087 [==============================] - 5s 1ms/step - loss: 0.4957 - acc: 0.7771 - val_loss: 0.4807 - val_acc: 0.7759
Epoch 98/200
3087/3087 [==============================] - 5s 1ms/step - loss: 0.4937 - acc: 0.7749 - val_loss: 0.4808 - val_acc: 0.7772
Epoch 99/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4919 - acc: 0.7742 - val_loss: 0.4806 - val_acc: 0.7772
Epoch 100/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4943 - acc: 0.7775 - val_loss: 0.4807 - val_acc: 0.7772
Epoch 101/200
3087/3087 [==============================] - 5s 2ms/step - loss: 0.4926 - acc: 0.7758 - val_loss: 0.4807 - val_acc: 0.7759
Epoch 102/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4968 - acc: 0.7765 - val_loss: 0.4806 - val_acc: 0.7746
Epoch 103/200
3087/3087 [==============================] -

Epoch 156/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4985 - acc: 0.7726 - val_loss: 0.4808 - val_acc: 0.7798
Epoch 157/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4975 - acc: 0.7771 - val_loss: 0.4808 - val_acc: 0.7798
Epoch 158/200
3087/3087 [==============================] - 5s 1ms/step - loss: 0.5027 - acc: 0.7736 - val_loss: 0.4809 - val_acc: 0.7798
Epoch 159/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4953 - acc: 0.7732 - val_loss: 0.4809 - val_acc: 0.7798
Epoch 160/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4965 - acc: 0.7755 - val_loss: 0.4807 - val_acc: 0.7785
Epoch 161/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4949 - acc: 0.7797 - val_loss: 0.4807 - val_acc: 0.7798
Epoch 162/200
3087/3087 [==============================] - 4s 1ms/step - loss: 0.4958 - acc: 0.7784 - val_loss: 0.4808 - val_acc: 0.7798
Epoch 163/200
3087/3087 [================

3087/3087 [==============================] - 6s 2ms/step - loss: 0.5265 - acc: 0.7570 - val_loss: 0.4958 - val_acc: 0.7539
Epoch 16/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5290 - acc: 0.7609 - val_loss: 0.4946 - val_acc: 0.7617
Epoch 17/50
3087/3087 [==============================] - 5s 2ms/step - loss: 0.5305 - acc: 0.7577 - val_loss: 0.4950 - val_acc: 0.7668
Epoch 18/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5270 - acc: 0.7629 - val_loss: 0.4981 - val_acc: 0.7642
Epoch 19/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5222 - acc: 0.7574 - val_loss: 0.4915 - val_acc: 0.7655
Epoch 20/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5304 - acc: 0.7551 - val_loss: 0.4952 - val_acc: 0.7617
Epoch 21/50
3087/3087 [==============================] - 6s 2ms/step - loss: 0.5172 - acc: 0.7519 - val_loss: 0.4914 - val_acc: 0.7539
Epoch 22/50
3087/3087 [==============================] - 5s 2ms/ste

In [ ]:
0.8297